# Mi cadena de markov

## Bibliotecas

In [1]:
# Warnings ---------------------------------------------------------------------------------------------------

import warnings
warnings.filterwarnings("ignore")

# Lectura y manipulación de datos ----------------------------------------------------------------------------

import pandas as pd

# Descomposición espectral -----------------------------------------------------------------------------------

import numpy as np
from numpy.linalg import eig, inv


## Lectura de la base de datos

In [2]:
data = pd.read_parquet('data/tec_estocasticos.parquet', engine='pyarrow')

In [3]:
data

,periodo,cliente_id,material_id,tipo_cliente
0,05-2022,4894.0,22.0,Distribuidor
1,05-2022,4769.0,17.0,Distribuidor
2,05-2022,4823.0,227.0,Distribuidor
3,08-2022,4816.0,340.0,Distribuidor
4,08-2022,4888.0,270.0,Distribuidor
...,...,...,...,...
6973895,05-2022,1816.0,1861.0,Farmacia
6973896,05-2022,1725.0,1373.0,Hospital
6973897,05-2022,588.0,2249.0,Hospital
6973898,05-2022,565.0,191.0,Hospital


In [4]:
data['periodo'] = pd.to_datetime(data['periodo'])
data.sort_values(by='periodo', inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop('index', axis = 1,inplace=True)
data

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1,2021-01-01,782.0,2043.0,Hospital
2,2021-01-01,303.0,1352.0,Hospital
3,2021-01-01,314.0,1577.0,Hospital
4,2021-01-01,122.0,1504.0,Hospital
...,...,...,...,...
6955591,2023-09-01,4406.0,317.0,Hospital
6955592,2023-09-01,587.0,2321.0,Hospital
6955593,2023-09-01,587.0,1359.0,Hospital
6955594,2023-09-01,2421.0,341.0,Hospital


### Tipos de Cliente

In [5]:
data['tipo_cliente'].unique()

array(['Hospital', 'Farmacia', 'Distribuidor', 'Otro'], dtype=object)

## Función para obtener la matriz de transición de un cliente comprando un determinado producto

In [5]:
def matriz_transicion(tipo_cliente, cliente_id, material_id):
    
    # Subdataset
    cliente_tipo = data.loc[data['tipo_cliente'] == tipo_cliente]
    pruducto = cliente_tipo.loc[cliente_tipo['material_id'] == material_id]
    id_cliente = pruducto.loc[pruducto['cliente_id'] == cliente_id]
    id_cliente.reset_index(inplace = True)
    id_cliente.drop('index', axis = 1, inplace = True)
    
    # Estados: Compró o no compró
    
    t = [0]

    for x in range(0, len(id_cliente['periodo'])-2):
        if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
            t.append(0)
        else:
            for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)-1):
                t.append(1)
            t.append(0)
    
    t_1 = []

    for x in range(0, len(id_cliente['periodo'])-1):
        if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
            t_1.append(0)
        else:
            for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)-1):
                t_1.append(1)
            t_1.append(0)
            

    estados = pd.DataFrame()

    estados['t'] = t
    estados['t_1'] = t_1
    
    Xt = estados['t'][0:-1].reset_index(drop=True).rename('X_t')
    Xt_1 = estados['t_1'][1::].reset_index(drop=True).rename('X_t+1')
    
    new_data=pd.concat((Xt, Xt_1), axis=1)
    
    matriz_transicion = new_data.groupby('X_t').value_counts(normalize=True).unstack(level='X_t+1')
    matriz_transicion= matriz_transicion.fillna(0)
    
    return matriz_transicion

In [7]:
 matriz_transicion('Distribuidor', 4769.0, 17.0)

X_t+1,0,1
X_t,,
0,0.928571,0.071429
1,0.666667,0.333333


In [24]:
matriz_transicion =  matriz_transicion('Distribuidor', 4769.0, 17.0)

## Descomposición espectral

In [8]:
## Descomposición espectral

import numpy as np
from numpy.linalg import eig, inv
Lambda, Q = eig(matriz_transicion)
print("Los eigenvalores de P son:", Lambda)
print("Los eigenvectores de P son:", Q)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [ ]:
Q_1 = inv(Q)
Lambda = np.diag(Lambda)
PP = np.matmul(np.matmul(Q, Lambda), Q_1)
PP.round(decimals = 4)

## Consultar probabilidad en tiempo n

In [6]:
def pasos(n):
    Lambda_n = Lambda**n
    P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)
    
    df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
    df.rename(columns = {0:'Compra'}, inplace = True)
    df.rename(columns = {1:'No Compra'}, inplace = True)
    return df

In [38]:
pasos(5)

,Compra,No Compra
Compra,0.9033,0.0967
No compra,0.9021,0.0979


In [39]:
pasos(10)

,Compra,No Compra
Compra,0.9032,0.0968
No compra,0.9032,0.0968


In [40]:
pasos(15)

,Compra,No Compra
Compra,0.9032,0.0968
No compra,0.9032,0.0968


In [13]:
data.loc[data['tipo_cliente'] == 'Hospital']

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1,2021-01-01,782.0,2043.0,Hospital
2,2021-01-01,303.0,1352.0,Hospital
3,2021-01-01,314.0,1577.0,Hospital
4,2021-01-01,122.0,1504.0,Hospital
...,...,...,...,...
6955591,2023-09-01,4406.0,317.0,Hospital
6955592,2023-09-01,587.0,2321.0,Hospital
6955593,2023-09-01,587.0,1359.0,Hospital
6955594,2023-09-01,2421.0,341.0,Hospital


## Dropdown

In [8]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

In [14]:
# Opciones

h = data.loc[data['tipo_cliente'] == 'Hospital']

tipo_cliente = list(h['tipo_cliente'].unique())

cliente_id = list(h['cliente_id'].unique()) 

material_id = list(h['material_id'].unique()) 

pasos_t = list(range(1,51))

# Botones

dropdown_tipo_cliente = widgets.Dropdown(
    options=tipo_cliente,
    description="tipo_cliente:"
)

dropdown_cliente_id = widgets.Dropdown(
    options=cliente_id,
    description="cliente_id:"
)

dropdown_material_id = widgets.Dropdown(
    options=material_id,
    description="material_id:"
)

dropdown_pasos = widgets.Dropdown(
    options=pasos_t,
    description="pasos:"
)


# Función para la visualización con parámetros
def valores(tipo_cliente, cliente_id, material_id, pasos_t):
    
    Lambda, Q = eig(matriz_transicion(tipo_cliente, cliente_id, material_id))
    Q_1 = inv(Q)
    Lambda = np.diag(Lambda)
    PP = np.matmul(np.matmul(Q, Lambda), Q_1)
    
    Lambda_n = Lambda**pasos_t
    P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)
    
    df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
    df.rename(columns = {0:'Compra'}, inplace = True)
    df.rename(columns = {1:'No Compra'}, inplace = True)
    
    df
    return print(df)

widgets.interactive(valores, 
                    tipo_cliente=dropdown_tipo_cliente, 
                    cliente_id=dropdown_cliente_id,
                    material_id=dropdown_material_id,
                    pasos_t = dropdown_pasos)

interactive(children=(Dropdown(description='tipo_cliente:', options=('Hospital',), value='Hospital'), Dropdown…